In [1]:
'''
找就位道、计算倒凹、填倒凹
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/9月下旬项目重开"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

obj_file_path = "150颗0806新数据/3.obj"
txt_file_path = "多分类预处理0929/预处理结果文档/3.txt"

当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\9月下旬项目重开


In [2]:
# 1.准备数据
import chardet
import numpy as np

def detect_encoding(file_path):
    '''检测文件的encoding方式'''
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result['encoding']
        
def load_obj_file(file_path, encoding):
    '''读取obj文件中的点云坐标，输出是list'''
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return vertices, faces

def insert_midpoint_points(vertices, faces, minimum_vertices_number=3000):
    if len(vertices) >= minimum_vertices_number:
        print('原始点云的点数足够多')
        return vertices, faces

    else:
        temp = 0
        while len(vertices) < minimum_vertices_number:
            edge_to_midpoint = {}
            new_points = []
            new_faces = []
            vertex_offset = len(vertices)
    
            for face in faces:
                # Compute midpoints for each edge
                midpoints = []
                for i in range(len(face)):
                    edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
                    if edge not in edge_to_midpoint:
                        midpoint = np.mean([vertices[edge[0]], vertices[edge[1]]], axis=0)
                        edge_to_midpoint[edge] = vertex_offset + len(new_points)
                        new_points.append(midpoint.tolist())
                    midpoints.append(edge_to_midpoint[edge])
                
                # Original vertices
                v0, v1, v2 = face
                # Midpoints
                m0, m1, m2 = midpoints

                # Create four new faces
                new_faces.append([v0, m0, m2])
                new_faces.append([v1, m1, m0])
                new_faces.append([v2, m2, m1])
                new_faces.append([m0, m1, m2])
    
            vertices.extend(new_points)
            faces = new_faces
            temp += 1

        print(f'使用了 {temp} 轮插值来让点云点数满足要求')
        
    return vertices, faces

def load_labeled_point_cloud(file_path):
    '''读取点云数据和标签'''
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)

In [3]:
import open3d as o3d
import numpy as np
from tqdm import tqdm
from scipy.spatial import KDTree
from shapely.geometry import Polygon

# Möller–Trumbore算法检测射线与三角形相交
def ray_triangle_intersection(ray_origin, ray_direction, triangle):
    EPSILON = 1e-6  # 用于避免浮点数误差

    vertex0, vertex1, vertex2 = triangle
    
    # 计算三角形的两个边
    edge1 = vertex1 - vertex0
    edge2 = vertex2 - vertex0

    # 计算叉积
    h = np.cross(ray_direction, edge2)
    a = np.dot(edge1, h)

    if -EPSILON < a < EPSILON:
        return False, None  # 射线与三角形平行

    f = 1.0 / a
    s = ray_origin - vertex0
    u = f * np.dot(s, h)

    if u < 0.0 or u > 1.0:
        return False, None

    q = np.cross(s, edge1)
    v = f * np.dot(ray_direction, q)

    if v < 0.0 or u + v > 1.0:
        return False, None

    # 计算 t 以确定交点位置
    t = f * np.dot(edge2, q)

    if t > EPSILON:
        intersection_point = ray_origin + ray_direction * t
        return True, intersection_point

    return False, None  # 射线在三角形平面上但没有相交


# 根据就位道方向进行三角形投影，检查投影重合
def detect_overlapping_triangles_by_direction(point_cloud, triangles, ray_direction):
    overlapping_triangles = set()

    # 创建一个与就位道方向垂直的平面
    normal = ray_direction / np.linalg.norm(ray_direction)  # 归一化的就位道方向
    projection_matrix = np.eye(3) - np.outer(normal, normal)  # 投影矩阵

    # 存储投影后的三角形
    projected_triangles = []

    for tri_idx in triangles:
        triangle = point_cloud[tri_idx]
        projected_triangle = np.dot(triangle, projection_matrix.T)  # 投影到垂直于就位道的平面上
        polygon_2d = Polygon([(v[0], v[1]) for v in projected_triangle])  # 只保留 x 和 y 维度
        projected_triangles.append(polygon_2d)

    # 检查每对投影是否重叠
    for i in range(len(projected_triangles)):
        for j in range(i + 1, len(projected_triangles)):
            if projected_triangles[i].intersects(projected_triangles[j]):
                overlapping_triangles.add(i)
                overlapping_triangles.add(j)

    return overlapping_triangles


# 使用KD-tree和射线投影检测单个方向上的倒凹区域
def detect_undercut_in_direction_with_kdtree(point_cloud, crown_indices, triangles, ray_direction, kdtree, overlapping_triangles, k_nearest=10):
    undercut_points = []

    # 遍历 crown_indices，不使用进度条，避免过多更新导致卡死
    for i, point in enumerate(point_cloud[crown_indices]):
        is_undercut = False

        # 使用KD-tree找到距离该点最近的三角形顶点索引
        distances, indices = kdtree.query(point, k=k_nearest)

        # 只检测来自重合部分的三角形
        for idx in indices:
            if idx not in overlapping_triangles:
                continue

            tri_idx = triangles[idx]
            triangle = point_cloud[tri_idx]
            
            # 使用Möller–Trumbore算法判断射线与三角形相交
            intersect, _ = ray_triangle_intersection(point, ray_direction, triangle)
            
            if intersect:
                is_undercut = True
                break  # 一旦有相交就可以停止检测

        # 如果检测到相交，说明当前点是倒凹点
        if is_undercut:
            undercut_points.append(point)

    return np.array(undercut_points)


# 生成多个方向，遍历所有方向找到最优方向
def find_optimal_insertion_direction_with_kdtree(point_cloud, crown_indices, triangles, kdtree, n_directions=500, k_nearest=10):
    min_undercut_points = None
    optimal_direction = None
    min_undercut_count = float('inf')

    # 随机生成多个方向
    directions = np.random.randn(n_directions, 3)
    directions = directions / np.linalg.norm(directions, axis=1)[:, np.newaxis]  # 归一化

    # 使用 tqdm 创建方向遍历进度条，只显示方向处理进度
    for direction in tqdm(directions, desc="Processing directions"):
        # 1. 根据就位道方向投影三角形，检测投影后重叠的三角形
        overlapping_triangles = detect_overlapping_triangles_by_direction(point_cloud, triangles, direction)

        # 2. 使用KD-tree进行倒凹检测
        undercut_points = detect_undercut_in_direction_with_kdtree(
            point_cloud, crown_indices, triangles, direction, kdtree, overlapping_triangles, k_nearest
        )

        # 3. 找到最优方向（最少倒凹点）
        if len(undercut_points) < min_undercut_count:
            min_undercut_count = len(undercut_points)
            min_undercut_points = undercut_points
            optimal_direction = direction

    return optimal_direction, min_undercut_points, min_undercut_count


# 可视化倒凹点和点云
def visualize_undercut(point_cloud, undercut_points, ray_direction):
    # 创建Open3D点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    
    # 创建用于表示倒凹区域的点云
    undercut_pcd = o3d.geometry.PointCloud()
    undercut_pcd.points = o3d.utility.Vector3dVector(undercut_points)
    
    # 将点云颜色设为浅蓝色，倒凹区域设为红色
    pcd.paint_uniform_color([0.6, 0.8, 1.0])
    undercut_pcd.paint_uniform_color([1.0, 0.0, 0.0])
    
    # 可视化就位道方向
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
    
    # 创建用于表示就位道方向的虚线箭头
    ray_origin = np.array([0, 0, 0])
    ray_end = ray_origin + ray_direction * 10  # 延长射线以便可视化
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector([ray_origin, ray_end]),
        lines=o3d.utility.Vector2iVector([[0, 1]])
    )
    line_set.paint_uniform_color([0, 0, 0])  # 设置为黑色
    
    # 显示点云、倒凹和方向箭头
    o3d.visualization.draw_geometries([pcd, undercut_pcd, line_set, axis])


# 测试代码
if __name__ == "__main__":
    # 1. 加载或生成点云和三角面片数据
    obj_encoding = detect_encoding(obj_file_path)
    obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
    checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces, 5000)  # 对点数小于5000的点云做插值加密
    vertices, labels = load_labeled_point_cloud(txt_file_path)
    crown_idx = [idx for idx in range(len(labels)) if labels[idx] != 1]

    point_cloud = vertices
    triangles = np.array(faces)
    crown_indices = np.array(crown_idx)
    
    # 2. 构建KD-Tree
    kdtree = KDTree(point_cloud)

    # 3. 遍历多个方向并找到最优的插入方向
    optimal_direction, optimal_undercut_points, undercut_count = find_optimal_insertion_direction_with_kdtree(
        point_cloud, crown_indices, triangles, kdtree, n_directions=500, k_nearest=10
    )

    print(f"Optimal insertion direction: {optimal_direction}")
    print(f"Number of undercut points: {undercut_count}")

    # 4. 可视化倒凹区域和点云
    visualize_undercut(point_cloud, optimal_undercut_points, optimal_direction)


使用了 1 轮插值来让点云点数满足要求


Processing directions:   0%|                                                                   | 0/500 [04:20<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import open3d as o3d
import numpy as np
from tqdm import tqdm
from scipy.spatial import KDTree
from shapely.geometry import Polygon

# Möller–Trumbore算法检测射线与三角形相交
def ray_triangle_intersection(ray_origin, ray_direction, triangle):
    EPSILON = 1e-6  # 用于避免浮点数误差

    vertex0, vertex1, vertex2 = triangle
    
    # 计算三角形的两个边
    edge1 = vertex1 - vertex0
    edge2 = vertex2 - vertex0

    # 计算叉积
    h = np.cross(ray_direction, edge2)
    a = np.dot(edge1, h)

    if -EPSILON < a < EPSILON:
        return False, None  # 射线与三角形平行

    f = 1.0 / a
    s = ray_origin - vertex0
    u = f * np.dot(s, h)

    if u < 0.0 or u > 1.0:
        return False, None

    q = np.cross(s, edge1)
    v = f * np.dot(ray_direction, q)

    if v < 0.0 or u + v > 1.0:
        return False, None

    # 计算 t 以确定交点位置
    t = f * np.dot(edge2, q)

    if t > EPSILON:
        intersection_point = ray_origin + ray_direction * t
        return True, intersection_point

    return False, None  # 射线在三角形平面上但没有相交


# 根据就位道方向进行三角形投影，检查投影重合
def detect_overlapping_triangles_by_direction(point_cloud, triangles, ray_direction):
    overlapping_triangles = set()

    # 创建一个与就位道方向垂直的平面
    normal = ray_direction / np.linalg.norm(ray_direction)  # 归一化的就位道方向
    projection_matrix = np.eye(3) - np.outer(normal, normal)  # 投影矩阵

    # 存储投影后的三角形
    projected_triangles = []

    for tri_idx in triangles:
        triangle = point_cloud[tri_idx]
        projected_triangle = np.dot(triangle, projection_matrix.T)  # 投影到垂直于就位道的平面上
        polygon_2d = Polygon([(v[0], v[1]) for v in projected_triangle])  # 只保留 x 和 y 维度
        projected_triangles.append(polygon_2d)

    # 检查每对投影是否重叠
    for i in range(len(projected_triangles)):
        for j in range(i + 1, len(projected_triangles)):
            if projected_triangles[i].intersects(projected_triangles[j]):
                overlapping_triangles.add(i)
                overlapping_triangles.add(j)

    return overlapping_triangles


# 使用KD-tree和射线投影检测单个方向上的倒凹区域
def detect_undercut_in_direction_with_kdtree(point_cloud, crown_indices, triangles, ray_direction, kdtree, overlapping_triangles, k_nearest=10):
    undercut_points = []

    # 使用 tqdm 创建进度条，遍历 crown_indices
    for i, point in tqdm(enumerate(point_cloud[crown_indices]), total=len(crown_indices), desc="Processing points"):
        is_undercut = False

        # 使用KD-tree找到距离该点最近的三角形顶点索引
        distances, indices = kdtree.query(point, k=k_nearest)

        # 只检测来自重合部分的三角形
        for idx in indices:
            if idx not in overlapping_triangles:
                continue

            tri_idx = triangles[idx]
            triangle = point_cloud[tri_idx]
            
            # 使用Möller–Trumbore算法判断射线与三角形相交
            intersect, _ = ray_triangle_intersection(point, ray_direction, triangle)
            
            if intersect:
                is_undercut = True
                break  # 一旦有相交就可以停止检测

        # 如果检测到相交，说明当前点是倒凹点
        if is_undercut:
            undercut_points.append(point)

    return np.array(undercut_points)


# 生成多个方向，遍历所有方向找到最优方向
def find_optimal_insertion_direction_with_kdtree(point_cloud, crown_indices, triangles, kdtree, n_directions=500, k_nearest=10):
    min_undercut_points = None
    optimal_direction = None
    min_undercut_count = float('inf')

    # 随机生成多个方向
    directions = np.random.randn(n_directions, 3)
    directions = directions / np.linalg.norm(directions, axis=1)[:, np.newaxis]  # 归一化

    # 使用 tqdm 创建方向遍历进度条
    for direction in tqdm(directions, desc="Processing directions"):
        # 1. 根据就位道方向投影三角形，检测投影后重叠的三角形
        overlapping_triangles = detect_overlapping_triangles_by_direction(point_cloud, triangles, direction)

        # 2. 使用KD-tree进行倒凹检测
        undercut_points = detect_undercut_in_direction_with_kdtree(
            point_cloud, crown_indices, triangles, direction, kdtree, overlapping_triangles, k_nearest
        )

        # 3. 找到最优方向（最少倒凹点）
        if len(undercut_points) < min_undercut_count:
            min_undercut_count = len(undercut_points)
            min_undercut_points = undercut_points
            optimal_direction = direction

    return optimal_direction, min_undercut_points, min_undercut_count


# 可视化倒凹点和点云
def visualize_undercut(point_cloud, undercut_points, ray_direction):
    # 创建Open3D点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    
    # 创建用于表示倒凹区域的点云
    undercut_pcd = o3d.geometry.PointCloud()
    undercut_pcd.points = o3d.utility.Vector3dVector(undercut_points)
    
    # 将点云颜色设为浅蓝色，倒凹区域设为红色
    pcd.paint_uniform_color([0.6, 0.8, 1.0])
    undercut_pcd.paint_uniform_color([1.0, 0.0, 0.0])
    
    # 可视化就位道方向
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
    
    # 创建用于表示就位道方向的虚线箭头
    ray_origin = np.array([0, 0, 0])
    ray_end = ray_origin + ray_direction * 10  # 延长射线以便可视化
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector([ray_origin, ray_end]),
        lines=o3d.utility.Vector2iVector([[0, 1]])
    )
    line_set.paint_uniform_color([0, 0, 0])  # 设置为黑色
    
    # 显示点云、倒凹和方向箭头
    o3d.visualization.draw_geometries([pcd, undercut_pcd, line_set, axis])


# 测试代码
if __name__ == "__main__":
    # 1. 加载或生成点云和三角面片数据
    obj_encoding = detect_encoding(obj_file_path)
    obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
    checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces, 5000)  # 对点数小于5000的点云做插值加密
    vertices, labels = load_labeled_point_cloud(txt_file_path)
    crown_idx = [idx for idx in range(len(labels)) if labels[idx]!=1]

    point_cloud = vertices
    triangles = np.array(faces)
    crown_indices = np.array(crown_idx)
    
    # 2. 构建KD-Tree
    kdtree = KDTree(point_cloud)

    # 3. 遍历多个方向并找到最优的插入方向
    optimal_direction, optimal_undercut_points, undercut_count = find_optimal_insertion_direction_with_kdtree(
        point_cloud, crown_indices, triangles, kdtree, n_directions=500, k_nearest=10
    )

    print(f"Optimal insertion direction: {optimal_direction}")
    print(f"Number of undercut points: {undercut_count}")

    # 4. 可视化倒凹区域和点云
    visualize_undercut(point_cloud, optimal_undercut_points, optimal_direction)
